# Mixed Density Network Training

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import tensorflow as tf
import keras
import h5py
import utils
import numpy as np

In [ ]:
import genconfig
from train_MDN import train_nn, mixture_density_loss

In [ ]:
from collections import namedtuple
import yaml
Option = namedtuple("MyStruct", "training_input training_output validation_fraction outFile network_type \
                    config structure learning_rate regularizer epochs")

# Two Particle Training

Create Config file, options are : 'number','pos1','pos2','pos3', 'error1x', 'error1y', 'error2x', 'error2y', 'error3x', 'error3y'  
Below is example of pos2

In [ ]:
!python genconfig.py --type pos2 > pos2config.json

Now with generated config run train_nn which stores training history in text file in training_output directory

In [ ]:
options = Option(
    training_input='NNinput_pos2_training.h5',
    training_output='Trained',
    validation_fraction=0.1,
    outFile='pos2',
    network_type='2particle',
    config='pos2config.json',
    structure=[100, 80, 50],
    learning_rate=0.0001,
    regularizer=0.0001,
    epochs=1,
)

print('loading config from: ', options.config)
config = open(options.config, 'r')
yamlConfig = yaml.load(config, Loader=yaml.FullLoader)
yamlConfig

In [ ]:
model_2, history_2 = train_nn(
                        options.training_input,
                        options.training_output,
                        options.validation_fraction,
                        options.outFile,
                        options.network_type,
                        options.config,
                        options.structure,
                        options.learning_rate,
                        options.regularizer,
                        options.epochs,
                        #args.momentum,
                        #args.batch,
                        ##args.min_epochs,
                        #args.max_epochs,
                        ##args.patience_increase,
                        ##args.threshold,
                        #args.min_delta,
                        #args.patience,
                        #args.profile,
                        #args.nbworkers,
                        #args.save_all,
                        )

Saving the model

In [ ]:
import datetime

In [ ]:
training_output = 'Trained'
outFile = 'pos2'
output_location = training_output+'/'+outFile+'_'+datetime.datetime.now().strftime('%m_%d_%H%M')

In [ ]:
model_2.save(output_location)

In [ ]:
new_model = tf.keras.models.load_model(output_location, custom_objects={'abs' : tf.keras.backend.abs,
                                                                        'loss' : mixture_density_loss(nb_components=1)})

Tensorflow 2.x features method model.save() which creates model file that stores both the weights and architecture. Loading the model from this format requires only custom objects like the above activation and loss functions to be provided. That means it includes the config, optimizer, and weights of the model. Older Tensorflow/keras model saving format required saving a weights .h5 file as well as the config and rebuilding the model and its architecture from that, which still requires the custom objects as well.

In [ ]:
model.get_config()

### Learning Curve Two Particle Training

In [ ]:
training_loss = np.loadtxt('Trained/pos2_06_04_0556_training_pos2.txt')
validation_loss = np.loadtxt('Trained/pos2_06_04_0556_validation_pos2.txt')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_learning_curve(train_loss, val_loss, fileName, outputDir):
    plt.figure(figsize=(7,5))
    
    plt.plot(train_loss, color='b', linewidth=0.5)
    plt.plot(val_loss, color='r', linewidth=0.5)
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xlim(0,250)
    plt.ylim(0, 0.9)
    plt.legend(['training set','validation set'])
    plt.savefig(outputDir+'/'+fileName+'_Loss.pdf')
    plt.show()
    plt.close()

In [ ]:
plot_learning_curve(training_loss, validation_loss, 'pos2', 'Trained')

# One Particle Training

In [ ]:
!python genconfig.py --type pos1 > pos1config.json

In [ ]:
options = Option(
    training_input='NNinput_pos1_training.h5',
    training_output='Trained',
    validation_fraction=0.1,
    outFile='pos1',
    network_type='1particle',
    config='pos1config.json',
    structure=[100, 50, 50],
    learning_rate=0.0001,
    regularizer=0.0001,
    epochs=250,
)

print('loading config from: ', options.config)
config = open(options.config, 'r')
yamlConfig = yaml.load(config, Loader=yaml.FullLoader)
yamlConfig

In [ ]:
model_1, history_1 = train_nn(
                        options.training_input,
                        options.training_output,
                        options.validation_fraction,
                        options.outFile,
                        options.network_type,
                        options.config,
                        options.structure,
                        options.learning_rate,
                        options.regularizer,
                        options.epochs,
                        #args.momentum,
                        #args.batch,
                        ##args.min_epochs,
                        #args.max_epochs,
                        ##args.patience_increase,
                        ##args.threshold,
                        #args.min_delta,
                        #args.patience,
                        #args.profile,
                        #args.nbworkers,
                        #args.save_all,
                        )

# Three Particle Training

In [ ]:
!python genconfig.py --type pos3 > pos3config.json

In [ ]:
options = Option(
    training_input='NNinput_pos3_training.h5',
    training_output='Trained',
    validation_fraction=0.1,
    outFile='pos3',
    network_type='3particle',
    config='pos3config.json',
    structure=[100, 80, 50],
    learning_rate=0.0001,
    regularizer=0.0001,
    epochs=250,
)

print('loading config from: ', options.config)
config = open(options.config, 'r')
yamlConfig = yaml.load(config, Loader=yaml.FullLoader)
yamlConfig

In [ ]:
model_3, history_3 = train_nn(
                        options.training_input,
                        options.training_output,
                        options.validation_fraction,
                        options.outFile,
                        options.network_type,
                        options.config,
                        options.structure,
                        options.learning_rate,
                        options.regularizer,
                        options.epochs,
                        #args.momentum,
                        #args.batch,
                        ##args.min_epochs,
                        #args.max_epochs,
                        ##args.patience_increase,
                        ##args.threshold,
                        #args.min_delta,
                        #args.patience,
                        #args.profile,
                        #args.nbworkers,
                        #args.save_all,
                        )

# Testing Inference

In [ ]:
model_2p = tf.keras.models.load_model('Trained/pos2_06_04_0556', custom_objects={'abs' : tf.keras.backend.abs,
                                                                        'loss' : mixture_density_loss(nb_components=1)})

In [ ]:
with h5py.File('NNinput_pos3_training.h5', 'r') as hf:  
    #x_train = hf['train_data_x'][:]
    #y_train = hf['train_data_y'][:]
    x_valid = hf['valid_data_x'][:]
    y_valid = hf['valid_data_y'][:]

In [ ]:
branches = utils.get_data_config_names('pos2config.json', meta=False)
print(branches)

In [ ]:
y_pred = model_3.predict(x_valid)

In [ ]:
pred_mean = np.hstack((y_pred[0][:,1:3], y_pred[1][:,1:3], y_pred[2][:,1:3]))
pred_prec = np.hstack((y_pred[0][:,3:5], y_pred[1][:,3:5], y_pred[2][:,3:5]))

In [ ]:
pred_mean - y_valid

In [ ]:
y_valid

In [ ]:
pred_mean = np.hstack((y_pred[0][:,1:3], y_pred[1][:,1:3]))
pred_mean[:4]

In [ ]:
y_valid[:4]

In [ ]:
pred_prec = np.hstack((y_pred[0][:,3:5], y_pred[1][:,3:5]))
pred_prec[:4]

In [ ]:
min(y_pred[0][:,0])

In [ ]:
y_pred

In [ ]:
x_valid[0]

In [ ]:
x_test_1 = tf.reshape(x_valid[0], [1,60])
x_test_1.shape

In [ ]:
y_pred_1 = model_2p.predict(x_test_1)
y_pred_1

In [ ]:
y_pred_1[0][0]

In [ ]:
y_pred_1[1][0]

In [ ]:
y_valid[0]

In [ ]:
y_valid[1]